In [6]:
import pandas as pd
from sklearn.metrics import accuracy_score

In [3]:
ROOT = "/home/lazye/Documents/ufrgs/mcs/clip/clip-bias-explore/\
fair-face-classification"
RESULTS_PATH = ROOT + "/data/results"

# original paper preds df with race_gender preds
opd_df = pd.read_csv(RESULTS_PATH+"/original_paper_preds.csv")

# top synm preds df
topk_df = pd.read_csv(RESULTS_PATH+"/top_k_synms.csv")

# avg sum synm preds df
avg_df = pd.read_csv(RESULTS_PATH+"/avg_sum_synms.csv") 


In [10]:
def filter_df(df, col, val):
    return df[df[col] == val]

## Original CLIP Paper Predictions Analysis

In [4]:
opd_df.drop(columns=['service_test'], inplace=True)
print(opd_df.head(1))
print('\ntransforming gender_preds to binary classes...\n')
opd_df['gender_preds'] = opd_df['gender_preds'].map(lambda x: x.split('_')[-1])
print(opd_df.head(1))


        file  age gender        race          gender_preds
0  val/1.jpg  3-9   Male  East Asian  Southeast Asian_Male

transforming gender_preds to binary classes...

        file  age gender        race gender_preds
0  val/1.jpg  3-9   Male  East Asian         Male


In [5]:
print("Validation set info:\n")
print("-"*15)
race_count = opd_df.race.value_counts()
race_percent = opd_df.race.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
gender_count = opd_df.gender.value_counts()
gender_percent = opd_df.gender.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

val_info_race = {"race_count": race_count,
                 "race_percent": race_percent}

val_info_race_df = pd.DataFrame(val_info_race)
print("Race info:\n")
print(val_info_race_df.head())

val_info_gender = {"gender_count": gender_count,
                   "gender_percent": gender_percent}

val_info_gender_df = pd.DataFrame(val_info_gender)
print("\nGender info:\n")
print(val_info_gender_df.head())
print("-"*15)


Validation set info:

---------------
Race info:

                 race_count race_percent
race                                    
White                  2085        19.0%
Latino_Hispanic        1623        14.8%
Black                  1556        14.2%
East Asian             1550        14.2%
Indian                 1516        13.8%

Gender info:

        gender_count gender_percent
gender                             
Male            5792          52.9%
Female          5162          47.1%
---------------


## Synonyms Prediction Analysis

### Top 1 Synms

In [27]:
def gender_acc(df):
    return accuracy_score(df['gender'], df['gender_preds'])

def acc_by_col(df, col):
    for unique in df[col].unique():
        col_df = df[df[col] == unique]
        col_acc = gender_acc(col_df)
        print(f"{unique} predictions accuracy: {round(col_acc, 2)}")

def gen_report(df, pred_name):
    print(f"# {pred_name} info:\n")
    print("-"*20)

    print("\n## General Accuracy")
    gen_acc = gender_acc(df)
    gen_miss = df[df['gender'] != df['gender_preds']]
    print(f"Prediction error count: {len(gen_miss)}")
    print(f"Prediction accuracy score: {round(gen_acc, 2)}")

    print("\n## Accuracy by gender")
    male_df = filter_df(df, 'gender', 'Male')
    male_acc = gender_acc(male_df)
    female_df = filter_df(df, 'gender', 'Female')
    female_acc = gender_acc(female_df)
    print(f"Male: {round(male_acc, 2)}")
    print(f"Female: {round(female_acc, 2)}")

    print("\n## Accuracy by race")
    acc_by_col(df, 'race')

    print("\n## Accuracy by age")
    acc_by_col(df, 'age')

In [29]:
gen_report(topk_df, "Top 1 Synonym Prediction")

# Top 1 Synonym Prediction info:

--------------------

## General Accuracy
Prediction error count: 541
Prediction accuracy score: 0.95

## Accuracy by gender
Male: 0.96
Female: 0.93

## Accuracy by race
East Asian predictions accuracy: 0.95
White predictions accuracy: 0.95
Latino_Hispanic predictions accuracy: 0.96
Southeast Asian predictions accuracy: 0.95
Black predictions accuracy: 0.92
Indian predictions accuracy: 0.96
Middle Eastern predictions accuracy: 0.98

## Accuracy by age
3-9 predictions accuracy: 0.87
50-59 predictions accuracy: 0.98
30-39 predictions accuracy: 0.98
20-29 predictions accuracy: 0.97
more than 70 predictions accuracy: 0.97
40-49 predictions accuracy: 0.98
10-19 predictions accuracy: 0.91
60-69 predictions accuracy: 0.96
0-2 predictions accuracy: 0.82


In [30]:
gen_report(opd_df, "Original Paper Predictions")

# Original Paper Predictions info:

--------------------

## General Accuracy
Prediction error count: 534
Prediction accuracy score: 0.95

## Accuracy by gender
Male: 0.96
Female: 0.94

## Accuracy by race
East Asian predictions accuracy: 0.95
White predictions accuracy: 0.96
Latino_Hispanic predictions accuracy: 0.96
Southeast Asian predictions accuracy: 0.95
Black predictions accuracy: 0.91
Indian predictions accuracy: 0.95
Middle Eastern predictions accuracy: 0.98

## Accuracy by age
3-9 predictions accuracy: 0.87
50-59 predictions accuracy: 0.97
30-39 predictions accuracy: 0.98
20-29 predictions accuracy: 0.97
more than 70 predictions accuracy: 0.95
40-49 predictions accuracy: 0.98
10-19 predictions accuracy: 0.91
60-69 predictions accuracy: 0.96
0-2 predictions accuracy: 0.83
